# Validation of the PyPSA-Earth stats

## Description
This task aims to develop such notebook that:
- takes as input the files from folders from pypsa-earth: `results/{scenarios}/stats.csv` (see PR Create statistics #579). In the meantime, data is loaded from `notebooks/validation/temp_stats_csv/stats_merged_20_3_23.csv`
- loads open data on power systems across the world
- Creates plots to perform the validation
Plots and tables shall have different aggregation levels (e.g. demand for a continent)

Create statistics for:
- demand (See `demand_validation.ipynb`)
- installed capacity by technology (compare with: IRENA, ...)
- renewable sources  (compare with: IRENA, ...)
- network characteristics (length of lines for example, https://wiki.openmod-initiative.org/wiki/Transmission_network_datasets)

Plots:
- Compare the statistics of the PyPSA-Earth model with open data

## Public data sources collection
These sources could be helpful:
- [ENTSO-E](https://transparency.entsoe.eu/generation/r2/installedGenerationCapacityAggregation/show)
- [IRENA](https://www.irena.org/data-and-statistics), not working
- [IEA](https://www.iea.org/data-and-statistics)
    - Electricity demand: https://www.iea.org/data-and-statistics/data-product/world-energy-balances-highlights
- [WEC](https://www.worldenergy.org/statistics/), not working
- [WRI](https://www.wri.org/resources/data-sets)
- [UN](https://unstats.un.org/unsd/snaama/)
- [WBG](https://datacatalog.worldbank.org/dataset/world-development-indicators)
- [OECD](https://data.oecd.org/)
- [Eurostat](https://ec.europa.eu/eurostat/data/database)
- [EIA](https://www.eia.gov/outlooks/aeo/data/browser/)
- [Enerdata](https://www.enerdata.net/research/)
- [BP](https://www.bp.com/en/global/corporate/energy-economics/statistical-review-of-world-energy.html)
- [USAID](https://www.usaid.gov/what-we-do/energy/global-energy-database), Single countries only?

https://www.usaid.gov/powerafrica/nigeria


## TODO
- DONE: Include continent analysis with country converter coco
- DONE: Continent `Asia` shows high ror and low hydro in PyPSA-Earth, but low ror and high hydro in IRENA. Why? Technology mismatch?
- Include stats on how many of the countries of a specific continent are in the PyPSA-Earth model, to better compare continental data. coco could be helpful here. Is this necessary?
- Fossil fuels as oil, gas, coal are often summed up as 'Fossil fuels n.e.s' in Europe which means 'not elsewhere specified' in IRENA data. How to deal with this?

## Questions
- Where is the demand in `stats.csv` ?

## Preparation

### Import packages

In [ ]:
import logging
import os
import sys

import pypsa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import country_converter as coco

logger = logging.getLogger(__name__)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 70)

In [ ]:
nice_names = {
    "nuclear": "Nuclear",
    "oil": "Oil",
    "onwind": "Onshore wind",
    #"ror": "Run of river",
    "solar": "Solar PV",
    "hydro": "Hydro",
    "gas": "Gas",
    "coal_and_lignite": "Coal",
}

### Set main directory to root folder

In [ ]:
# change current directory
module_path = os.path.abspath(os.path.join('../../../')) # To import helpers

if module_path not in sys.path:
    sys.path.append(module_path+"/pypsa-earth/scripts")
    
from _helpers import sets_path_to_root, country_name_2_two_digits, two_digits_2_name_country

sets_path_to_root("documentation")

### Load stats data (obtained from pypsa-earth)

In [ ]:
# Read it with multilevel column names. Make sure that the country index "NA" is not recognized as NaN
stats = pd.read_csv("notebooks/validation/temp_stats_csv/stats_merged_07_04_23.csv", index_col=0, header=[0,1], keep_default_na=False, na_values="")

### Load public data

In [ ]:
EXAMPLE_URL="https://pxweb.irena.org/pxweb/en/IRENASTAT/IRENASTAT__Power%20Capacity%20and%20Generation/ELECCAP_2022_cycle2.px/"

In [ ]:
# Read the data "https://pxweb.irena.org/pxweb/en/IRENASTAT/IRENASTAT__Power%20Capacity%20and%20Generation/ELECCAP_2022_cycle2.px/"
# TODO can we download the data directly?
irena_eleccap = pd.read_csv("notebooks/validation/temp_irena/ELECCAP_20230314-165057.csv", encoding="latin-1", skiprows=2)

# Replace ".." in the dataframe with NaN
irena_eleccap = irena_eleccap.replace("..", np.nan)

# Change dtype of column "Installed electricity capacity by country/area (MW)" to float
irena_eleccap["Installed electricity capacity by country/area (MW)"] = irena_eleccap["Installed electricity capacity by country/area (MW)"].astype(float)

In [ ]:
# Combine ongrid and offgrid
irena_eleccap = irena_eleccap.groupby(["Country/area", "Year", "Technology"]).sum(numeric_only=True).reset_index() #"Technology", "Installed electricity capacity by country/area (MW)"

# Delete the column "Year" since it is not needed anymore
irena_eleccap = irena_eleccap.drop(columns=["Year"])

In [ ]:
# Check data for a single country
irena_eleccap[irena_eleccap["Country/area"] == "Germany"].head(5)

## Validation

### Installed capacity by technology

In [ ]:
# Define the technologies which should be compared
techs = ["CCGT", "OCGT", "nuclear", "onwind", "solar", "ror",  "hydro", "oil", "coal", "lignite"]

# Select rule "add_electricity" and their techs 
stats_capacities = stats["add_electricity"].loc[:, (techs)]

# Add continent at the beginning of the dataframe
stats_capacities.insert(0, "continent", coco.convert(names = stats_capacities.index, src = 'ISO2', to = 'continent'))

# Replace NaN with zeros
stats_capacities = stats_capacities.fillna(0)

In [ ]:
# Combine CCGT and OCGT to "gas"
stats_capacities["gas"] = stats_capacities["CCGT"] + stats_capacities["OCGT"]
stats_capacities["coal_and_lignite"] = stats_capacities["coal"] + stats_capacities["lignite"]
stats_capacities["hydro"] = stats_capacities["ror"] + stats_capacities["hydro"]
del stats_capacities["CCGT"] 
del stats_capacities["OCGT"] # TODO write in one line
del stats_capacities["coal"] 
del stats_capacities["lignite"]
del stats_capacities["ror"]

In [ ]:
# Rename to nice plotting names
stats_capacities.rename(columns=nice_names, inplace=True)

In [ ]:
stats_capacities.head()

#### Uniform technology names and dataframe structure

In [ ]:
# Create dict to match the technology names of stats_capacities and irena eleccap
uniform_names = {"Solar photovoltaic": "solar",
        "Onshore wind energy": "onwind",
        #"Offshore wind energy": "offwind",
        "Renewable hydropower": "hydro",
        "Nuclear": "nuclear",
        "Oil": "oil",
        "Natural gas": "gas",
        "Mixed Hydro Plants": "ror", # TODO Is this correct? Check IRENA    
        "Coal and peat": "coal_and_lignite",
        }

In [ ]:
# Rename the technologies in irena_eleccap to match the names in stats_capacities using the dict names
irena_eleccap["Technology"] = irena_eleccap["Technology"].replace(uniform_names)

# Transform technologies to columns and have the countries as index
irena_eleccap = irena_eleccap.pivot_table(index=["Country/area"], columns="Technology", values="Installed electricity capacity by country/area (MW)")
# Reset name of columns
irena_eleccap.columns.name = None

# Combine the columns ror and hydro and name them hydro
irena_eleccap["hydro"] = irena_eleccap["ror"] + irena_eleccap["hydro"]
del irena_eleccap["ror"]

In [ ]:
# Change the index of irena_eleccap to two digit country name using the function country_name_2_two_digits()
irena_eleccap.index = irena_eleccap.index.map(country_name_2_two_digits)

In [ ]:
# Add continent at the beginning of the dataframe
irena_eleccap.insert(0, "continent", coco.convert(names = irena_eleccap.index, src = 'ISO2', to = 'continent'))

# Rename to nice plotting names
irena_eleccap.rename(columns=nice_names, inplace=True)

#### Plot country comparison

In [ ]:
def plot_barplot(data_stats, data_irena, area):
    # Plot a barplot to compare the technologies of the two dataframes irena_eleccap and stats_capacities

    # Get the index values
    index = np.arange(len(data_stats))
    barWidth = 0.3

    # Create a barplot
    plt.figure(figsize=(10, 4))
    plt.bar(index - barWidth/2, data_stats/1e3, color=['g'], alpha=1, edgecolor='white', width=barWidth) # convert from MW to GW
    plt.bar(index + barWidth/2, data_irena/1e3, color=['g'], alpha=0.3, edgecolor='white', width=barWidth) # convert from MW to GW

    # Enhance graph
    plt.xticks(index, data_stats.index)
    plt.ylabel("Capacity in GW")
    plt.legend(["PyPSA-Earth", "IRENA"], loc='upper left', ncol=1)

    plt.grid(axis='y', alpha=0.5)
    plt.title(f"Electric capacity in {area}")

    # Show graphic & save it
    plt.savefig(f"notebooks/validation/temp_results/el_cap_{area}", bbox_inches='tight') # TODO add save path
    plt.show()
    
    return

In [ ]:
def is_country_or_continent(area_name):

    # Check if the area name is a valid ISO-3166 country code

    area_name = coco.convert(names=area_name, to='name_short')
    if area_name != "not found":
        return "country"
    else:
        return "continent" # TODO all invalid names are continents, this should be improved


In [ ]:
def area_selection(area, stats_capacities, irena_eleccap):

    _stats_capacities = stats_capacities.copy()
    _irena_eleccap = irena_eleccap.copy()

    if is_country_or_continent(area) == "continent":

        # Group the data by continent if area is "continent"
        _irena_eleccap = _irena_eleccap.groupby("continent").sum()
        _stats_capacities = _stats_capacities.groupby("continent").sum()

    elif is_country_or_continent(area) == "country":
        try:
            _irena_eleccap.drop(columns=["continent"], inplace=True)
            _stats_capacities.drop(columns=["continent"], inplace=True)
        except:
            pass

    # Select the data for the area
    data_irena = _irena_eleccap.loc[area][_stats_capacities.loc[area].index]
    data_stats = _stats_capacities.loc[area]

    return data_stats, data_irena

### Plot comparison (both country or continent possible)

In [ ]:
area = "MA"
data_stats, data_irena = area_selection(area, stats_capacities, irena_eleccap)
plot_barplot(data_stats, data_irena, area)

In [ ]:
# Plot and save all continents
areas = stats_capacities.continent.unique()
areas = areas[areas != "not found"]
for area in areas:
    data_stats, data_irena = area_selection(area, stats_capacities, irena_eleccap)
    plot_barplot(data_stats, data_irena, area)
    if area == "Europe":
        print("Fossil fuels as oil, gas, coal are often summed up as 'Fossil fuels n.e.s' in Europe which means 'not elsewhere specified' in IRENA data.")

### Demand

In [ ]:
stats.head()

In [ ]:
# Select rule "add_electricity" and their techs 
stats_demand = stats["add_electricity"].loc[:, "demand"]
stats_demand_solve = stats["solve_network"].loc[:, "demand"]

In [ ]:
# Create a dataframe with the demand of add_electricity and solve_network
stats_demand = pd.concat([stats["add_electricity"].loc[:, "demand"], stats["solve_network"].loc[:, "demand"]], axis=1)
stats_demand.columns = ["demand_add_el", "demand_solve"]

In [ ]:
# Plot a boxplot to compare the difference in percent between the demand in "stats_demand" and "stats_demand_solve"
diff = ((stats_demand["demand_solve"] - stats_demand["demand_add_el"])/stats_demand["demand_add_el"]*100).dropna()
plt.figure(figsize=(6, 4))
plt.boxplot(diff)
plt.ylabel("Demand reduction in %")
plt.title("Demand reduction from 'add_electricity' to 'solve_network'")
plt.grid(axis='y', alpha=0.5)
plt.xticks([1], ["Countries"])
plt.show()

In [ ]:
diff.describe()

In [ ]:
# Add continent at the beginning of the dataframe
stats_demand.insert(0, "continent", coco.convert(names = stats_demand.index, src = 'ISO2', to = 'continent'))

In [ ]:
stats_demand.head()

IEA

In [ ]:
# Read the data from the excel file
iea_web = pd.read_excel("notebooks/validation/temp_IEA/World Energy Balances Highlights 2022.xlsx", sheet_name="TimeSeries_1971-2021", skiprows=1, index_col=0)

In [ ]:
# filter the data, only "Total final consumption (PJ)" in the column "Flow" and "Electricity" in the column "Product" is required
iea_el_demand = iea_web[(iea_web["Flow"] == "Total final consumption (PJ)") & (iea_web["Product"] == "Electricity")]

# Get electricity demand of 2020 (most recent one available)
iea_el_demand = pd.DataFrame(iea_el_demand[2020]) * 0.277777778 #convert PJ to TWh 

In [ ]:
# Change index entry "Non-OECD Asia (including China)" to "Non-OECD Asia (including C)" to avoid regex to match "China"
iea_el_demand.rename(index={"Non-OECD Asia (including China)": "Non-OECD Asia (including C)"}, inplace=True)

# Change country name to two digit country code and keep the old index where country_name_2_two_digits() returns "not found"
old_index = iea_el_demand.index.tolist()
new_index = iea_el_demand.index.map(country_name_2_two_digits) .tolist()

for i in range(len(new_index)):
    if new_index[i] == "not found":
        new_index[i] = old_index[i]

iea_el_demand.index = new_index

In [ ]:
iea_el_demand.head()

### Networks

In [ ]:
# Select rule "add_electricity" and their techs 
stats_network = stats["base_network"].loc[:, ["lines_length", "lines_capacity"]]

In [ ]:
# Add continent at the beginning of the dataframe
stats_network.insert(0, "continent", coco.convert(names = stats_network.index, src = 'ISO2', to = 'continent'))

In [ ]:
stats_network.head()

GridKit

In [ ]:
# Get GridKit data https://zenodo.org/record/47317#.ZBw1KvaZM-U. Manually downloaded and extracted.
gridkit_europe = pd.read_csv("notebooks/validation/temp_gridkit/gridkit_euorpe/gridkit_europe-highvoltage-links.csv")
gridkit_northamerica = pd.read_csv("notebooks/validation/temp_gridkit/gridkit_north_america/gridkit_north_america-highvoltage-links.csv")

In [ ]:
gridkit_europe.head()

In [ ]:
gridkit_europe_length = gridkit_europe["length_m"].sum() / 1e3 # convert from m to km
gridkit_northamerica_length = gridkit_northamerica["length_m"].sum() / 1e3 # convert from m to km

In [ ]:
gridkit_europe_length

In [ ]:
gridkit_northamerica_length